# 🛑 Desarrollo a realizar

**Requisitos**

  1. Utilizar la clase Nodo explicada en clase con los métodos y atributos vistos en los ejercicios prácticos.

  2. Utiliza una matriz (sin librerías) de al menos 10x10.

  3. Resolver el problema empleando las versiones Graph de BFS, DFS, Greedy y A*.

  4. Define [explícitamente](https://dle.rae.es/expl%C3%ADcitamente) un orden de visita en el caso de los algoritmos no informados.

  5. En la representación del problema han de existir obstáculos.

  6. Por cada iteración del algoritmo muestra <font color='purple'>los nodos expandidos</font>, la frontera.
  
  7. Finalmente la ruta encontrada y el coste.
  
  8. Se han de relizar al menos 3 búsquedas.
  
  9. **<font color="red">No es posible utilizar librerías.</font>**
  
**Consideraciones deseables**
  * Desarrolla las versiones Tree de los algoritmos.

**🤖Requisitos extra**

* Búsquedas: Utilizar dos versiones de A* y Greedy con 2 heurísticas.


# 1 Búsquedas

## a) Problema a resolver 
Somos un concesionario de coches y queremos enviar coches según la demanda, pero por el mejor camino.

In [ ]:
matriz_ciudades = [
    ["Vitoria-Gasteiz", "Bilbao",      "Santander", "Logroño", "Burgos", "A Coruña", "Lugo", "Pontevedra", "Orense", "Santiago"],
    ["X",             "San Sebastián", "X",         "León",       "X", "Zamora", "Salamanca", "X", "X", "Valladolid"],
    ["X",              "Pamplona",     "X",           "Soria", "Segovia", "X", "Avila", "Madrid", "Guadalajara", "Malaga"],
    ["Zaragoza",         "Zestoa",    "Lleida",     "Huesca",     "X", "Castelo", "Toledo", "X", "Cuenca", "Valencia"],
    ["Tarragona",         "X",     "Barcelona", "Girona", "Huelva", "Castey", "Castellón", "X", "Alicante", "Murcia"],
    ["Albacete", "X", "Teruel", "Jaén", "Ciudad Real", "Córdoba", "Jaén", "X", "Cartagena", "Elche"],
    ["Badajoz", "Cáceres", "X", "X", "X", "X", "X", "X", "Granada", "Almería"],
    ["X", "Sevilla", "X", "X", "X", "X", "X", "X", "X", "Cadiz"],
    ["X", "Marbella", "X", "X", "X", "X", "X", "X", "X", "Ibiza"],
    ["Santa Cruz", "Las Palmas", "X", "X", "X", "X", "X", "Menorca", "Mallorca", "Paris"]
]


## b) Representación de estados



In [ ]:
class Nodo:
    def __init__(self, padre =None, profundidad= 0,accion = None,
                 estado = None, coste = None, hn = None, fn = None, posicion = None):
      self._padre = padre
      self._profundidad = profundidad
      self._accion = accion
      self._estado = estado
      self._coste = coste
      self._hn = hn
      self._fn = fn
      self._posicion = posicion # hemos añadido este atributo para que cuando insertemos el
      # el nodo inicial sepamos las posiciones del array y no tener que llamar
      # en la función de expandir una función para saber su posición.

    def get_padre(self):
      return self._padre

    def get_profundidad(self):
      return self._profundidad

    def get_accion(self):
      return self._accion

    def get_estado(self):
      return self._estado

    def get_coste(self):
      return self._coste

    # heuristica
    def get_hn(self):
      return self._hn
    # heuristica + coste acumulado
    def get_fn(self):
      return self._fn

    def get_posicion(self):
      return self._posicion

    # Métodos "set" para actualizar los valores de los atributos
    def set_padre(self, padre):
      self._padre = padre

    def set_profundidad(self, profundidad):
      self._profundidad = profundidad

    def set_accion(self, accion):
      self._accion = accion

    def set_estado(self, estado):
      self._estado = estado

    def set_coste(self,coste):
      self._coste = coste

    def set_hn(self,hn):
      self._hn = hn

    def set_fn(self,fn):
      self._fn = fn

    # SET PARA SETEAR LA POSICIÓN DEL NODO PADRE
    def set_posicion(self, posicion):
      self._posicion = posicion



## c) Código

### Funciones Comunes


    Estas funciones son comunes entre los algoritmos informados o no informados, lo hacemos para tener el código más limpio y tratar de emplear el menor número de celdas.



### Función Posición Inicial

    Con esta función conseguimos la posición del eje x y y del array, para luego poder expandir los hijos correctamente. Como a la hora de realizar las búsquedas empleamos un String, facilitando así la búsqueda al usuario, se necesita saber al menos la posición del nodo origen para luego recorrer el array.

    Esta función devuelve la posición del eje x e y.

In [ ]:
def posicion_inicial(city):
    for x in range(len(matriz_ciudades)):

        for y in range(len(matriz_ciudades[x])):

            if matriz_ciudades[x][y] == city:

                print(x,y)
                posicion = x,y
                return posicion

### Función Ruta

    Con esta función lo que conseguimos es devolver la ruta desde el objetivo hasta el origen. Como en la función expandir hemos añadido al nodo padre, hay que recorrer los padres hasta llegar a None.
    Este será el nodo origen porque a la hora de Instanciar el primer nodo no escribimos el atributo padre, por lo que se le asigna el valor que se encuentra en el constructor de la clase Nodo.

In [ ]:
def fx_Camino(nodo_actual):

    ruta = []
    ruta.append(nodo_actual.get_estado())
    padre_node = nodo_actual.get_padre()

    # recorrer hasta llegar al origen
    while padre_node is not None:
        ruta.append(padre_node.get_estado())
        padre_node = padre_node.get_padre()
    return ruta


## C1) Código de NO informados


### Función comun --> Expand()

    Debido a que la función de expansión de DBS y DFS es la misma, emplearemos una vez la función expand para al igual que antes, tener el código limpio.

In [ ]:
def expand(node):
    # 1. Inicializar sucesores como una lista vacía
    sucesores = []

    # Obtener el estado actual del nodo
    posicion_actual = node._posicion
    x_fila, y_columna = posicion_actual
    coste = 0.1

    # Definimos los movimientos posibles
    posibles_movimientos = [
        ("Derecha", 0, 1),  # Moverse a la derecha
        ("Abajo", 1, 0),  # Moverse hacia abajo
        ("Izquierda", 0, -1),  # Moverse a la izquierda
        ("Arriba", -1, 0)  # Moverse hacia arriba
    ]

    # 2. FOR EACH applicable action on state of current node DO
    for accion, dx, dy in posibles_movimientos:
        nueva_fila = x_fila + dx
        nueva_columna = y_columna + dy
        # Verificar que la nueva posición está dentro de los límites
        if 0 <= nueva_fila < len(matriz_ciudades) and 0 <= nueva_columna < len(matriz_ciudades[0]):
            nuevo_estado = matriz_ciudades[nueva_fila][nueva_columna]

            # Verificar si el movimiento es válido (no hay obstáculo)
            if nuevo_estado != "X":
                # 2.1 Make new node
                nodo_hijo = Nodo()
                # 2.2 Sucessor's state
                nodo_hijo.set_estado(nuevo_estado)
                # 2.2 Sucessor's action
                nodo_hijo.set_accion(accion)  # Coordenadas como acción
                # 2.4 Sucessor's parent node = current node
                nodo_hijo.set_padre(node)
                # 2.5 Sucessor's path-cost
                nodo_hijo.set_coste(node.get_coste() + coste)  # Coste acumulado
                # 2.6 Sucessor's depth
                nodo_hijo.set_profundidad(node.get_profundidad() + 1 )
                # 2.x Atributo para saber la posición en el mapa
                nodo_hijo.set_posicion((nueva_fila, nueva_columna))  # Posición nueva
                # 2.7 Add sucessor node to Sucessors
                sucesores.append(nodo_hijo)
    # 3. RETURN Sucessor's
    return sucesores


### BFS - GraphSearch


In [ ]:
def graphsearch_bfs(start, goal):

    frontier = []
    nodos_visitados = set()
    found = False
    current_depth = 0

    # 1.  Instanciamos el primer nodo ( Estado, Padre, Coste, Profundidad)
    nodo = Nodo()
    nodo.set_coste(current_depth)
    nodo.set_estado(start)
    nodo.set_profundidad(0)

    posicion = posicion_inicial(start)
    nodo.set_posicion(posicion)

    print(
        f"El estado del nodo actual es : {nodo.get_estado()} con posición {nodo._posicion}"
    )

    # 2. Insert node into frontier
    frontier.append(nodo)
    nodos_visitados.add(nodo._estado)

    # Comprobamos si el nodo inicial es el nodo objetivo
    if nodo.get_estado() == goal:
        found = True
        print("Nodo encontrado")

    # 3. WHILE final state not found and frontier is not empty
    while not found and frontier:

        # 3.1 extraemos elemento de la frontera según algoritmo
        node = frontier.pop(0)

        print(f"El estado actual es {node.get_estado()}")

        # 3.2 comprobamos si el nodo extraído es el objetivo
        if node.get_estado() == goal:
            found = True
            print(f"El nodo encontrado es {node.get_estado()}")
            print(f"El nodo tiene una profundidad de {node.get_profundidad()}")

        # 3.3 Does't contain final state
        else:

            # 3.3.1 EXPAND node's state
            sucesores = expand(node)

            for sucesor in sucesores:
                if sucesor.get_estado() not in nodos_visitados:

                    # 3.3.2
                    frontier.append(sucesor)
                    nodos_visitados.add(sucesor.get_estado())
                    print(f"Añadido nodo con estado: {sucesor.get_estado()} y coste de {sucesor._coste}")

            # url https://www.geeksforgeeks.org/python-map-function/
            # Con esta línea imprimo los estados de los nodos de las fronteras

            #                         devuelvo una lista
            #                           Sintaxis mapa(función, iterable)
            #                       Con la función lambda permitimos  coger los estados de los nodos y devolverlos

            print("Frontera actual:", list(map(lambda nodo: nodo.get_estado(), frontier)))
            print(nodos_visitados)

        # 4. IF final state found
        if found == True:

            ruta_auxiliar = fx_Camino(node)

            # Invertir la ruta para que empiece desde el origen
            ruta_auxiliar.reverse()

            # Mostrar la ruta y el costo total
            print("Ruta:", " -> ".join(ruta_auxiliar))
            print("Costo total:", node.get_coste())

In [ ]:
start = 'Vitoria-Gasteiz'
goal = 'Albacete'

graphsearch_bfs(start, goal)

0 0
El estado del nodo actual es : Vitoria-Gasteiz con posición (0, 0)
El estado actual es Vitoria-Gasteiz
Añadido nodo con estado: Bilbao y coste de 0.1
Frontera actual: ['Bilbao']
{'Bilbao', 'Vitoria-Gasteiz'}
El estado actual es Bilbao
Añadido nodo con estado: Santander y coste de 0.2
Añadido nodo con estado: San Sebastián y coste de 0.2
Frontera actual: ['Santander', 'San Sebastián']
{'Bilbao', 'Santander', 'Vitoria-Gasteiz', 'San Sebastián'}
El estado actual es Santander
Añadido nodo con estado: Logroño y coste de 0.30000000000000004
Frontera actual: ['San Sebastián', 'Logroño']
{'San Sebastián', 'Vitoria-Gasteiz', 'Bilbao', 'Logroño', 'Santander'}
El estado actual es San Sebastián
Añadido nodo con estado: Pamplona y coste de 0.30000000000000004
Frontera actual: ['Logroño', 'Pamplona']
{'Pamplona', 'San Sebastián', 'Vitoria-Gasteiz', 'Bilbao', 'Logroño', 'Santander'}
El estado actual es Logroño
Añadido nodo con estado: Burgos y coste de 0.4
Añadido nodo con estado: León y coste de

### BFS - TreeSearch


In [ ]:
def treesearch_bfs(start, goal):

    frontier = []
    nodos_visitados = set()
    found = False
    current_depth = 0

    # 1.  Instanciamos el primer nodo ( Estado, Padre, Coste, Profundidad)
    nodo = Nodo()
    nodo.set_coste(current_depth)
    nodo.set_estado(start)
    nodo.set_profundidad(0)

    posicion = posicion_inicial(start)
    nodo.set_posicion(posicion)

    print(
        f"El estado del nodo actual es : {nodo.get_estado()} con posición {nodo._posicion}"
    )

    # 2. Insert node into frontier
    frontier.append(nodo)
    nodos_visitados.add(nodo._estado)

    # Comprobamos si el nodo inicial es el nodo objetivo
    if nodo.get_estado() == goal:
        found = True
        print("Nodo encontrado")

    # 3. WHILE final state not found and frontier is not empty
    while not found and frontier:

        # 3.1 extraemos elemento de la frontera según algoritmo
        node = frontier.pop(0)

        print(f"El estado actual es {node.get_estado()}")

        # 3.2 comprobamos si el nodo extraído es el objetivo
        if node.get_estado() == goal:
            found = True
            print(f"El nodo encontrado es {node.get_estado()}")
            print(f"El nodo tiene una profundidad de {node.get_profundidad()}")

        # 3.3 Does't contain final state
        else:

            # 3.3.1 EXPAND node's state
            sucesores = expand(node)

            for sucesor in sucesores:
                frontier.append(sucesor)
                print(
                        f"Añadido nodo con estado: {sucesor.get_estado()} y coste de {sucesor._coste}"
                    )

            # url https://www.geeksforgeeks.org/python-map-function/
            # Con esta línea imprimo los estados de los nodos de las fronteras

            #                         devuelvo una lista
            #                           Sintaxis mapa(función, iterable)
            #                       Con la función lambda permitimos  coger los estados de los nodos y devolverlos

            print("Frontera actual:", list(map(lambda nodo: nodo.get_estado(), frontier)))

        # 4. IF final state found
        if found == True:

            ruta_auxiliar = fx_Camino(node)

            # Invertir la ruta para que empiece desde el origen
            ruta_auxiliar.reverse()

            # Mostrar la ruta y el costo total
            print("Ruta:", " -> ".join(ruta_auxiliar))
            print("Costo total:", node.get_coste())

In [ ]:
start = 'Vitoria-Gasteiz'
goal = 'Santiago'

#treesearch_bfs(start, goal)






### DFS - GraphSearch


In [ ]:
# Solución parcial
def dfs_graphsearch(start, goal):

    frontier = []
    nodos_visitados = set()
    found = False
    current_depth = 0

    # 1.  Instanciamos el primer nodo ( Estado, Padre, Coste, Profundidad)
    nodo = Nodo()
    nodo.set_coste(current_depth)
    nodo.set_estado(start)
    nodo.set_profundidad(0)

    posicion = posicion_inicial(start)
    nodo.set_posicion(posicion)

    print(
        f"El estado del nodo actual es : {nodo.get_estado()} con posición {nodo._posicion}"
    )

    # 2. Insert node into frontier
    frontier.append(nodo)
    nodos_visitados.add(nodo._estado)

    # Comprobamos si el nodo inicial es el nodo objetivo
    if nodo.get_estado() == goal:
        found = True
        print("Nodo encontrado")

    # 3. WHILE final state not found and frontier is not empty
    while not found and frontier:

        # 3.1 extraemos elemento de la frontera según algoritmo
        node = frontier.pop()

        print(f"El estado actual es {node.get_estado()}")

        # 3.2 comprobamos si el nodo extraído es el objetivo
        if node.get_estado() == goal:
            found = True
            # print("Nodo encontrado")
            print(f"El nodo encontrado es {node.get_estado()}")
            print(f"El nodo tiene una profundidad de {node.get_profundidad()}")

        # 3.3 Does't contain final state
        else:

            # 3.3.1 EXPAND node's state
            sucesores = expand(node)

            for sucesor in sucesores:
                #
                print(nodos_visitados)

                if sucesor.get_estado() not in nodos_visitados:

                    # 3.3.2
                    frontier.append(sucesor)
                    nodos_visitados.add(sucesor.get_estado())
                    print(
                        f"Añadido nodo con estado: {sucesor.get_estado()} y coste de {sucesor._coste}"
                    )

                # url https://www.geeksforgeeks.org/python-map-function/
                # Con esta línea imprimo los estados de los nodos de las fronteras

                #                         devuelvo una lista
                #                           Sintaxis mapa(función, iterable)
                #                       Con la función lambda permitimos  coger los estados de los nodos y devolverlos
                print("Frontera actual:",list(map(lambda nodo: nodo.get_estado(), frontier)))

        # 4. IF final state found
        if found == True:

            ruta_auxiliar = fx_Camino(node)

            # Invertir la ruta para que empiece desde el origen
            ruta_auxiliar.reverse()

            # Mostrar la ruta y el costo total
            print("Ruta:", " -> ".join(ruta_auxiliar))
            print("Costo total:", node.get_coste())

In [ ]:
start = 'Vitoria-Gasteiz'
goal = 'Albacete'

dfs_graphsearch(start, goal)

0 0
El estado del nodo actual es : Vitoria-Gasteiz con posición (0, 0)
El estado actual es Vitoria-Gasteiz
{'Vitoria-Gasteiz'}
Añadido nodo con estado: Bilbao y coste de 0.1
Frontera actual: ['Bilbao']
El estado actual es Bilbao
{'Bilbao', 'Vitoria-Gasteiz'}
Añadido nodo con estado: Santander y coste de 0.2
Frontera actual: ['Santander']
{'Bilbao', 'Santander', 'Vitoria-Gasteiz'}
Añadido nodo con estado: San Sebastián y coste de 0.2
Frontera actual: ['Santander', 'San Sebastián']
{'Bilbao', 'Santander', 'Vitoria-Gasteiz', 'San Sebastián'}
Frontera actual: ['Santander', 'San Sebastián']
El estado actual es San Sebastián
{'Bilbao', 'Santander', 'Vitoria-Gasteiz', 'San Sebastián'}
Añadido nodo con estado: Pamplona y coste de 0.30000000000000004
Frontera actual: ['Santander', 'Pamplona']
{'Pamplona', 'San Sebastián', 'Vitoria-Gasteiz', 'Bilbao', 'Santander'}
Frontera actual: ['Santander', 'Pamplona']
El estado actual es Pamplona
{'Pamplona', 'San Sebastián', 'Vitoria-Gasteiz', 'Bilbao', 'S

### DFS - TreeSearch


In [ ]:
# Solución parcial
def dfs_treesearch(start, goal):

    frontier = []
    nodos_visitados = set()
    found = False
    current_depth = 0

    # 1.  Instanciamos el primer nodo ( Estado, Padre, Coste, Profundidad)
    nodo = Nodo()
    nodo.set_coste(current_depth)
    nodo.set_estado(start)
    nodo.set_profundidad(0)

    posicion = posicion_inicial(start)
    nodo.set_posicion(posicion)

    print(
        f"El estado del nodo actual es : {nodo.get_estado()} con posición {nodo._posicion}"
    )

    # 2. Insert node into frontier
    frontier.append(nodo)
    nodos_visitados.add(nodo._estado)

    # Comprobamos si el nodo inicial es el nodo objetivo
    if nodo.get_estado() == goal:
        found = True
        print("Nodo encontrado")

    # 3. WHILE final state not found and frontier is not empty
    while not found and frontier:

        # 3.1 extraemos elemento de la frontera según algoritmo
        node = frontier.pop()

        print(f"El estado actual es {node.get_estado()}")

        # 3.2 comprobamos si el nodo extraído es el objetivo
        if node.get_estado() == goal:
            found = True
            # print("Nodo encontrado")
            print(f"El nodo encontrado es {node.get_estado()}")
            print(f"El nodo tiene una profundidad de {node.get_profundidad()}")

        # 3.3 Does't contain final state
        else:

            # 3.3.1 EXPAND node's state
            sucesores = expand(node)

            for sucesor in sucesores:


                    # 3.3.2
                    frontier.append(sucesor)
                    print(
                        f"Añadido nodo con estado: {sucesor.get_estado()} y coste de {sucesor._coste}"
                    )

                # url https://www.geeksforgeeks.org/python-map-function/
                # Con esta línea imprimo los estados de los nodos de las fronteras

                #                         devuelvo una lista
                #                           Sintaxis mapa(función, iterable)
                #                       Con la función lambda permitimos  coger los estados de los nodos y devolverlos
                    print("Frontera actual:",list(map(lambda nodo: nodo.get_estado(), frontier)))

        # 4. IF final state found
        if found == True:

            ruta_auxiliar = fx_Camino(node)

            # Invertir la ruta para que empiece desde el origen
            ruta_auxiliar.reverse()

            # Mostrar la ruta y el costo total
            print("Ruta:", " -> ".join(ruta_auxiliar))
            print("Costo total:", node.get_coste())

In [ ]:
start = 'Vitoria-Gasteiz'
goal = 'Santiago'

#treesearch_dfs(start, goal)

## C2) Código de Informados

### Funciones comunes.

    Como Greedy y A* se basan en heurísticas, vamos a emplear la heurística Euclidiana y la distancia de Manhattan. A diferencia de la anterior función de expansión, a esta le tenemos que pasar la posición del nodo origen y la posicion del estado actual.

Funciones Matemáticas:

- Euclidiana: **d = √[(x2 – x1)2 + (y2 – y1)2]**

- Distancia Manhattan: **Σ|Ai – Bi|**

Urls:
- [Euclidiana](https://en.wikipedia.org/wiki/Euclidean_distance)
- [Distancia Manhattan](https://statologos.com/distancia-manhattan-python/?utm_source=chatgpt.com&utm_content=cmp-true)


In [ ]:
# Función Euclidiana
def euclidiana(origen, posicion_actual):
    # obtengo los valores x e y de cada nodo
    x1, y1 = origen
    x2, y2 = posicion_actual

    distancia = ((x2 - x1)**2 + (y2 - y1)**2)**0.5

    return distancia

In [ ]:
# Función Distancia de Manhattan
def distancia_manhattan(origen, posicion_actual):

    return abs(posicion_actual[0] - origen[0]) + abs(posicion_actual[1] - origen[1])


Sin embargo, sus funciones de expandir son diferentes :

- <font color='green' style='bold'>Greedy</font> emplea la fórmula de <font color='green'>**f(successor) = h(successor)**</font>
    
- <font color='red'>A*</font> emplea <font color='red'>**f(successor) = h(successor)+ g(successor)**</font>

## Greedy

### Función comun ---> Expand() Greedy


In [ ]:
def expand(node, posicion_inicial, heuristica):
    # 1. Inicializar sucesores como una lista vacía
    sucesores = []

    # Obtener el estado actual del nodo
    estado_actual = node.get_estado()
    posicion_actual = node._posicion
    x_fila, y_columna = posicion_actual
    coste = 0.1  # Coste fijo por movimiento (puede ajustarse)

    # Definimos los movimientos posibles
    posibles_movimientos = [
        ("Derecha", 0, 1),  # Moverse a la derecha
        ("Izquierda", 0, -1),  # Moverse a la izquierda
        ("Arriba", -1, 0),  # Moverse hacia arriba
        ("Abajo", 1, 0)  # Moverse hacia abajo
    ]

    # 2. FOR EACH applicable action on state of current node DO
    for accion, dx, dy in posibles_movimientos:
        nueva_fila = x_fila + dx
        nueva_columna = y_columna + dy
        # Verificar que la nueva posición está dentro de los límites
        if 0 <= nueva_fila < len(matriz_ciudades) and 0 <= nueva_columna < len(matriz_ciudades[0]):
            nuevo_estado = matriz_ciudades[nueva_fila][nueva_columna]

            # Verificar si el movimiento es válido (no hay obstáculo)
            if nuevo_estado != "X":
                # 2.1 Make new node
                nodo_hijo = Nodo()
                # 2.2 Sucessor's state
                nodo_hijo.set_estado(nuevo_estado)
                # 2.3 Sucessor's action
                nodo_hijo.set_accion(accion)
                # 2.4 Sucessor's parent node = current node
                nodo_hijo.set_padre(node)
                # 2.5 Sucessor's path-cost
                nodo_hijo.set_coste(node.get_coste() + coste)

                #  2.6 Heurística
                posicion_actual = nueva_fila, nueva_columna
                valor_hn = heuristica(posicion_inicial, posicion_actual)
                nodo_hijo.set_hn(valor_hn)
                # 2.7 Sucessor's depth
                nodo_hijo.set_profundidad(node.get_profundidad() + 1 )
                # 2.8 Atributo para saber la posición en el mapa
                nodo_hijo.set_posicion((nueva_fila, nueva_columna))  # Posición nueva
                # 2.9 Add sucessor node to Sucessors
                sucesores.append(nodo_hijo)
    # 3. RETURN Sucessor's
    return sucesores


### Greedy - GraphSearch

In [ ]:
#Solución parcial
def greedy_graphsearch(start, goal, heuristica):

    frontier=[]
    found = False
    current_depth = 0
    nodos_visitados = set()

    #1.  Instanciamos el primer nodo ( Estado, Padre, Coste, Profundidad)
    nodo = Nodo()
    nodo.set_coste(current_depth)
    nodo.set_estado(start)
    nodo.set_profundidad(0)

    posicion = posicion_inicial(start)
    nodo.set_posicion(posicion)


    print(f"El estado del nodo actual es : {nodo.get_estado()} con posición {nodo._posicion}")

    # 2. Insert node into frontier
    frontier.append(nodo)
    nodos_visitados.add(nodo._estado)


    # Comprobamos si el nodo inicial es el nodo objetivo
    if nodo.get_estado() == goal:
        found = True
        print("Nodo encontrado")

    # 3. WHILE final state not found and frontier is not empty
    while not found and frontier:

        # 3.1 extraemos elemento de la frontera según algoritmo
        node = frontier.pop(0)
        print(f"El estado actual es {node.get_estado()}")

        # 3.2 comprobamos si el nodo extraído es el objetivo
        if node.get_estado() == goal:
            found = True
            #print("Nodo encontrado")
            print(f"El nodo encontrado es {node.get_estado()}")
            print(f"El nodo tiene una profundidad de {node.get_profundidad()}")

        # 3.3 Does't contain final state
        else:
            # 3.3.1 EXPAND node's state
            sucesores = expand(node, nodo._posicion, heuristica)
            for sucesor in sucesores:
                if sucesor.get_estado() not in nodos_visitados:
                    # 3.3.2
                    frontier.append(sucesor)
                    nodos_visitados.add(sucesor._estado)
                    print(f"Se ha expandido al nodo con estado: {sucesor.get_estado()} y coste heusrístico de {sucesor._hn}")

                # 4.3.4
                frontier.sort(key=lambda nodo: nodo.get_hn())

            # url https://www.geeksforgeeks.org/python-map-function/
            # Con esta línea imprimo los estados de los nodos de las fronteras
            #                         devuelvo una lista
            print("Frontera actual:", list(map(lambda nodo: nodo.get_estado(), frontier)))



    # 4. IF final state found
    if found == True:

      ruta_auxiliar = fx_Camino(node)

      # Invertir la ruta para que empiece desde el origen
      ruta_auxiliar.reverse()

      # Mostrar la ruta y el costo total
      print("Ruta:", " -> ".join(ruta_auxiliar))
      print("Costo total:", node.get_coste())
      print("Costo total heurística:", node.get_hn())


In [ ]:
start = 'Vitoria-Gasteiz'
goal = 'Paris'
heuristica = euclidiana

greedy_graphsearch(start, goal, heuristica)

0 0
El estado del nodo actual es : Vitoria-Gasteiz con posición (0, 0)
El estado actual es Vitoria-Gasteiz
Se ha expandido al nodo con estado: Bilbao y coste heusrístico de 1.0
Frontera actual: ['Bilbao']
El estado actual es Bilbao
Se ha expandido al nodo con estado: Santander y coste heusrístico de 2.0
Se ha expandido al nodo con estado: San Sebastián y coste heusrístico de 1.4142135623730951
Frontera actual: ['San Sebastián', 'Santander']
El estado actual es San Sebastián
Se ha expandido al nodo con estado: Pamplona y coste heusrístico de 2.23606797749979
Frontera actual: ['Santander', 'Pamplona']
El estado actual es Santander
Se ha expandido al nodo con estado: Logroño y coste heusrístico de 3.0
Frontera actual: ['Pamplona', 'Logroño']
El estado actual es Pamplona
Se ha expandido al nodo con estado: Zestoa y coste heusrístico de 3.1622776601683795
Frontera actual: ['Logroño', 'Zestoa']
El estado actual es Logroño
Se ha expandido al nodo con estado: Burgos y coste heusrístico de 4.0


In [ ]:
start = 'Vitoria-Gasteiz'
goal = 'Paris'
heuristica = distancia_manhattan

greedy_graphsearch(start, goal, heuristica)

0 0
El estado del nodo actual es : Vitoria-Gasteiz con posición (0, 0)
El estado actual es Vitoria-Gasteiz
Se ha expandido al nodo con estado: Bilbao y coste heusrístico de 1
Frontera actual: ['Bilbao']
El estado actual es Bilbao
Se ha expandido al nodo con estado: Santander y coste heusrístico de 2
Se ha expandido al nodo con estado: San Sebastián y coste heusrístico de 2
Frontera actual: ['Santander', 'San Sebastián']
El estado actual es Santander
Se ha expandido al nodo con estado: Logroño y coste heusrístico de 3
Frontera actual: ['San Sebastián', 'Logroño']
El estado actual es San Sebastián
Se ha expandido al nodo con estado: Pamplona y coste heusrístico de 3
Frontera actual: ['Logroño', 'Pamplona']
El estado actual es Logroño
Se ha expandido al nodo con estado: Burgos y coste heusrístico de 4
Se ha expandido al nodo con estado: León y coste heusrístico de 4
Frontera actual: ['Pamplona', 'Burgos', 'León']
El estado actual es Pamplona
Se ha expandido al nodo con estado: Zestoa y co

### Greedy - TreeSearch

In [ ]:
def greedy_treesearch(start, goal, heuristica):

    frontier=[]
    found = False
    current_depth = 0
    nodos_visitados = set()

    #1.  Instanciamos el primer nodo ( Estado, Padre, Coste, Profundidad)
    nodo = Nodo()
    nodo.set_coste(current_depth)
    nodo.set_estado(start)
    nodo.set_profundidad(0)

    posicion = posicion_inicial(start)
    nodo.set_posicion(posicion)


    print(f"El estado del nodo actual es : {nodo.get_estado()} con posición {nodo._posicion}")

    # 2. Insert node into frontier
    frontier.append(nodo)
    nodos_visitados.add(nodo._estado)


    # Comprobamos si el nodo inicial es el nodo objetivo
    if nodo.get_estado() == goal:
        found = True
        print("Nodo encontrado")

    # 3. WHILE final state not found and frontier is not empty
    while not found and frontier:

        # 3.1 extraemos elemento de la frontera según algoritmo
        node = frontier.pop(0)
        print(f"El estado actual es {node.get_estado()}")

        # 3.2 comprobamos si el nodo extraído es el objetivo
        if node.get_estado() == goal:
            found = True
            #print("Nodo encontrado")
            print(f"El nodo encontrado es {node.get_estado()}")
            print(f"El nodo tiene una profundidad de {node.get_profundidad()}")

        # 3.3 Does't contain final state
        else:
            # 3.3.1 EXPAND node's state
            sucesores = expand(node, nodo._posicion, heuristica)
            for sucesor in sucesores:
                    # 3.3.2
                    frontier.append(sucesor)
                    print(f"Se ha expandido al nodo con estado: {sucesor.get_estado()} y coste heusrístico de {sucesor._hn}")

                # 4.3.4
                    frontier.sort(key=lambda nodo: nodo.get_hn())

            # url https://www.geeksforgeeks.org/python-map-function/
            # Con esta línea imprimo los estados de los nodos de las fronteras
            #                         devuelvo una lista
            print("Frontera actual:", list(map(lambda nodo: nodo.get_estado(), frontier)))



    # 4. IF final state found
    if found == True:

      ruta_auxiliar = fx_Camino(node)

      # Invertir la ruta para que empiece desde el origen
      ruta_auxiliar.reverse()

      # Mostrar la ruta y el costo total
      print("Ruta:", " -> ".join(ruta_auxiliar))
      print("Costo total:", node.get_coste())
      print("Costo total heurística:", node.get_hn())

In [ ]:
start = 'Vitoria-Gasteiz'
goal = 'San Sebastian'
heuristica = euclidiana

#greedy_treesearch(start, goal, heuristica)

## A*

### Función comun ---> Expand() Greedy
- <font color='red'>A*</font> emplea <font color='red'>**f(successor) = h(successor)+ g(successor)**</font>

In [ ]:
def expand(node, posicion_inicial, heuristica):
    # 1. Inicializar sucesores como una lista vacía
    sucesores = []

    # Obtener el estado actual del nodo
    estado_actual = node.get_estado()
    posicion_actual = node._posicion
    x_fila, y_columna = posicion_actual
    coste = 0.1  # Coste fijo por movimiento (puede ajustarse)

    # Definimos los movimientos posibles
    posibles_movimientos = [
        ("Derecha", 0, 1),  # Moverse a la derecha
        ("Izquierda", 0, -1),  # Moverse a la izquierda
        ("Arriba", -1, 0),  # Moverse hacia arriba
        ("Abajo", 1, 0)  # Moverse hacia abajo
    ]

    # 2. FOR EACH applicable action on state of current node DO
    for accion, dx, dy in posibles_movimientos:
        nueva_fila = x_fila + dx
        nueva_columna = y_columna + dy
        # Verificar que la nueva posición está dentro de los límites
        if 0 <= nueva_fila < len(matriz_ciudades) and 0 <= nueva_columna < len(matriz_ciudades[0]):
            nuevo_estado = matriz_ciudades[nueva_fila][nueva_columna]

            # Verificar si el movimiento es válido (no hay obstáculo)
            if nuevo_estado != "X":
                # 2.1 Make new node
                nodo_hijo = Nodo()
                # 2.2 Sucessor's state
                nodo_hijo.set_estado(nuevo_estado)
                # 2.3 Sucessor's action
                nodo_hijo.set_accion(accion)
                # 2.4 Sucessor's parent node = current node
                nodo_hijo.set_padre(node)
                # 2.5 Sucessor's path-cost
                nodo_hijo.set_coste(node.get_coste() + coste)

                #  2.6.1 Heurística
                posicion_actual = nueva_fila, nueva_columna
                valor_hn = heuristica(posicion_inicial, posicion_actual)
                nodo_hijo.set_hn(valor_hn)

                # 2.6.2 Heuristica + coste
                valor_fn =  valor_hn + node._coste
                nodo_hijo.set_fn(valor_fn)
                # 2.7 Sucessor's depth
                nodo_hijo.set_profundidad(node.get_profundidad() + 1 )
                # 2.8 Atributo para saber la posición en el mapa
                nodo_hijo.set_posicion((nueva_fila, nueva_columna))  # Posición nueva
                # 2.9 Add sucessor node to Sucessors
                sucesores.append(nodo_hijo)
    # 3. RETURN Sucessor's
    return sucesores


### A* GraphSearch

In [ ]:
def aestrella_graphsearch(start, goal, heuristica):

    frontier = []
    found = False
    current_depth = 0
    nodos_visitados = set()

    # 1.  Instanciamos el primer nodo ( Estado, Padre, Coste, Profundidad)
    nodo = Nodo()
    nodo.set_coste(current_depth)
    nodo.set_estado(start)
    nodo.set_profundidad(0)

    posicion = posicion_inicial(start)
    nodo.set_posicion(posicion)

    print(
        f"El estado del nodo actual es : {nodo.get_estado()} con posición {nodo._posicion}"
    )

    # 2. Insert node into frontier
    frontier.append(nodo)

    # Comprobamos si el nodo inicial es el nodo objetivo
    if nodo.get_estado() == goal:
        found = True
        print("Nodo encontrado")

    # 3. WHILE final state not found and frontier is not empty
    while not found and frontier:

        # 3.1 extraemos elemento de la frontera según algoritmo
        node = frontier.pop(0)

        print(f"El estado actual es {node.get_estado()}")

        # 3.2 comprobamos si el nodo extraído es el objetivo
        if node.get_estado() == goal:
            found = True
            # print("Nodo encontrado")
            print(f"El nodo encontrado es {node.get_estado()}")
            print(f"El nodo tiene una profundidad de {node.get_profundidad()}")

        # 3.3 Does't contain final state
        else:

            # 3.3.1 EXPAND node's state
            sucesores = expand(node, nodo._posicion, heuristica)

            for sucesor in sucesores:

                if sucesor.get_estado() not in nodos_visitados:

                    # 3.3.2
                    frontier.append(sucesor)
                    nodos_visitados.add(sucesor._estado)
                    print(
                        f"Se ha expandido del {node.get_estado()} al nodo con estado: {sucesor.get_estado()} y coste de {sucesor._coste}"
                    )

                print("Frontera actual:",list(map(lambda nodo: nodo.get_estado(), frontier)))

                frontier.sort(key=lambda nodo: nodo.get_hn())

        # 4. IF final state found
        if found == True:

            ruta_auxiliar = fx_Camino(node)

            # Invertir la ruta para que empiece desde el origen
            ruta_auxiliar.reverse()

            # Mostrar la ruta y el costo total
            print("Ruta:", " -> ".join(ruta_auxiliar))
            print("Costo total:", node.get_coste())
            print("Costo total  + heurística:", node.get_fn())

In [ ]:
start = 'Vitoria-Gasteiz'
goal = 'Paris'
heuristica = euclidiana

aestrella_graphsearch(start, goal, heuristica)

0 0
El estado del nodo actual es : Vitoria-Gasteiz con posición (0, 0)
El estado actual es Vitoria-Gasteiz
Se ha expandido del Vitoria-Gasteiz al nodo con estado: Bilbao y coste de 0.1
Frontera actual: ['Bilbao']
El estado actual es Bilbao
Se ha expandido del Bilbao al nodo con estado: Santander y coste de 0.2
Frontera actual: ['Santander']
Se ha expandido del Bilbao al nodo con estado: Vitoria-Gasteiz y coste de 0.2
Frontera actual: ['Santander', 'Vitoria-Gasteiz']
Se ha expandido del Bilbao al nodo con estado: San Sebastián y coste de 0.2
Frontera actual: ['Vitoria-Gasteiz', 'Santander', 'San Sebastián']
El estado actual es Vitoria-Gasteiz
Frontera actual: ['San Sebastián', 'Santander']
El estado actual es San Sebastián
Frontera actual: ['Santander']
Se ha expandido del San Sebastián al nodo con estado: Pamplona y coste de 0.30000000000000004
Frontera actual: ['Santander', 'Pamplona']
El estado actual es Santander
Se ha expandido del Santander al nodo con estado: Logroño y coste de 0

In [ ]:
start = 'Vitoria-Gasteiz'
goal = 'Paris'
heuristica = distancia_manhattan

aestrella_graphsearch(start, goal, heuristica)

0 0
El estado del nodo actual es : Vitoria-Gasteiz con posición (0, 0)
El estado actual es Vitoria-Gasteiz
Se ha expandido del Vitoria-Gasteiz al nodo con estado: Bilbao y coste de 0.1
Frontera actual: ['Bilbao']
El estado actual es Bilbao
Se ha expandido del Bilbao al nodo con estado: Santander y coste de 0.2
Frontera actual: ['Santander']
Se ha expandido del Bilbao al nodo con estado: Vitoria-Gasteiz y coste de 0.2
Frontera actual: ['Santander', 'Vitoria-Gasteiz']
Se ha expandido del Bilbao al nodo con estado: San Sebastián y coste de 0.2
Frontera actual: ['Vitoria-Gasteiz', 'Santander', 'San Sebastián']
El estado actual es Vitoria-Gasteiz
Frontera actual: ['Santander', 'San Sebastián']
El estado actual es Santander
Se ha expandido del Santander al nodo con estado: Logroño y coste de 0.30000000000000004
Frontera actual: ['San Sebastián', 'Logroño']
Frontera actual: ['San Sebastián', 'Logroño']
El estado actual es San Sebastián
Frontera actual: ['Logroño']
Se ha expandido del San Seba